In [2]:
# 1. Mengimpor pustaka yang diperlukan
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# 2. Memuat dataset wrinkle5.csv
data = pd.read_csv('hasil_deteksi_kerutan_total_pixel.csv')

# 3. Mengganti nama label sesuai dengan nama folder
label_mapping = {
    'wrinkle ringan': 'wrinkle_ringan',
    'wrinkle sedang': 'wrinkle_sedang',
    'wrinkle berat': 'wrinkle_berat'
}
data['label'] = data['label'].map(label_mapping)

# 4. Menentukan fitur (X) dan target (y) dari dataset
X = data.drop(columns=['label', 'image_idx'])
y = data['label']

# 5. Membagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 6. Standarisasi data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 7. Membuat model Logistic Regression dengan regularisasi dan penyesuaian class_weight
# Menyesuaikan parameter C untuk regularisasi dan menggunakan solver yang lebih stabil seperti 'liblinear' untuk ukuran data kecil
model = LogisticRegression(class_weight='balanced', random_state=42, C=0.5, solver='liblinear')

# 8. Melatih model dengan data latih
model.fit(X_train_scaled, y_train)

# 9. Memprediksi dengan data uji
y_pred = model.predict(X_test_scaled)
probabilitas = model.predict_proba(X_test_scaled)  # Mendapatkan probabilitas untuk kelas

# 10. Menampilkan hasil evaluasi model
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# 11. Menampilkan akurasi
accuracy = accuracy_score(y_test, y_pred)
print(f"Akurasi Model: {accuracy:.4f}")

# 12. Evaluasi menggunakan Cross-validation dengan Stratified K-Fold (lebih baik untuk dataset yang tidak seimbang)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cross_val_scores = cross_val_score(model, X_train_scaled, y_train, cv=skf)
print(f"Cross-validation scores: {cross_val_scores}")
print(f"Average cross-validation score: {cross_val_scores.mean():.4f}")

# 13. Menampilkan probabilitas dan confidence untuk data uji pertama (contoh)
for i in range(10):  # Menampilkan beberapa probabilitas
    print(f"Prediksi untuk Data Uji {i + 1}:")
    print(f"Prediksi: {y_pred[i]}")
    print(f"Probabilitas: {probabilitas[i]}")
    confidence = max(probabilitas[i])  # Confidence untuk kelas yang dipilih
    print(f"Confidence: {confidence:.4f}\n")


Confusion Matrix:
[[32  0  0]
 [ 0  3  7]
 [ 0  0 78]]

Classification Report:
                precision    recall  f1-score   support

 wrinkle_berat       1.00      1.00      1.00        32
wrinkle_ringan       1.00      0.30      0.46        10
wrinkle_sedang       0.92      1.00      0.96        78

      accuracy                           0.94       120
     macro avg       0.97      0.77      0.81       120
  weighted avg       0.95      0.94      0.93       120

Akurasi Model: 0.9417
Cross-validation scores: [0.96875    0.97916667 0.96875    0.98958333 0.97916667]
Average cross-validation score: 0.9771
Prediksi untuk Data Uji 1:
Prediksi: wrinkle_sedang
Probabilitas: [0.00248324 0.01528808 0.98222868]
Confidence: 0.9822

Prediksi untuk Data Uji 2:
Prediksi: wrinkle_ringan
Probabilitas: [8.25308059e-09 5.22954075e-01 4.77045917e-01]
Confidence: 0.5230

Prediksi untuk Data Uji 3:
Prediksi: wrinkle_berat
Probabilitas: [0.64663761 0.00085767 0.35250473]
Confidence: 0.6466

Prediksi 

In [3]:
import joblib

# Menyimpan scaler ke file
joblib.dump(scaler, 'scaler_model.pkl')
print("Scaler berhasil disimpan.")


Scaler berhasil disimpan.


In [4]:
import joblib

# Simpan model ke file
joblib.dump(model, 'logistic_regression_model.pkl')
print("Model telah disimpan sebagai logistic_regression_model.pkl")


Model telah disimpan sebagai logistic_regression_model.pkl
